<a href="https://colab.research.google.com/github/LarryDCJ/AC_Classifier/blob/master/Kaggle_TitanicCompetition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import os
import numpy as np #linear algebra tools
import pandas as pd # data processing and CSV manipulation (r/w)
drive.mount('/content/drive')
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/kaggle"

#changing the working directory
%cd /content/drive/MyDrive/kaggle
!pwd

In [ ]:
!kaggle competitions list

In [ ]:
!kaggle datasets list -s Titanic

In [ ]:
#User datasets
!kaggle datasets download [USER/FILENAME]

#I prefer this one
!kaggle competitions download -c [COMPETITION]

!unzip [FILENAME.zip]
!ls

https://towardsdatascience.com/how-to-search-and-download-data-using-kaggle-api-f815f7b98080

This is the link for Kaggle+Colab instructions.


In [ ]:
for dirname, _, filenames in os.walk('/kaggle/):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_data = pd.read_csv("train.csv")
train_data.head()

In [ ]:
test_data = pd.read_csv("test.csv")
test_data.head()

In [ ]:
#This code lets us know the % of women that actually survived.
#By grabbing all columns from the TRAIN dataset, we can divide the amount of surviving
#women by the total amount of women, leaving us with about 74.2%.

women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

In [ ]:
#This does the same with men, coming to about 18.9% (RIP fellas).

men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

Why do you think the numbers are so heavily favored for women?

Questioning your data and the simple metrics gleaned from the data can help guide deeper insight into the problems you have along with the solutions you're searching for.

To gain a better prediction, we can use the information from multiple columns rather than only basing our prediction on the sex of the passsengers.

To do this, we will implement a RANDOM FOREST MODEL. 


In [ ]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

This is how you submit a prediction to a competition.

In [ ]:
#!kaggle competitions submit -c [COMPETITION NAME] -f [FILE PATH]
!kaggle competitions submit -c Titanic -f '/content/drive/MyDrive/kaggle/my_submission.csv' -m "My first Random Forest Model Submission"

Now, we'll implement fast.ai instructions to work with tabular data. If you want to know more about fast.ai, check out their free MOOC here: https://course.fast.ai

In [ ]:
!pip install fastai
!pip install nbdev
import fastai
from fastai.tabular.all import *
from nbdev.showdoc import *

In [ ]:
import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/kaggle/'): 
  for filename in filenames: 
    print(os.path.join(dirname, filename))

In [ ]:
fastai.__version__

In [ ]:
df_train = pd.read_csv('train.csv')
print(df_train.shape, df_train.head())
df_train.info()

In [ ]:
df_test = pd.read_csv('test.csv')
print(df_test.shape, df_test.head())
df_test.info()